# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [2]:
name = "high_vs_random_with_med_low"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,Unnamed: 0,text_clean,label
0,0,helium network hnt cryptocoin iconlogo before ...,1
1,1,schuurshoeijmakers syndromemondo term id and l...,0
2,2,incompleteness disjoint types this issue was c...,0
3,3,reqmgr2 acdc creation via webpage naming needs...,0
4,4,writing a literal string to the record io chan...,1
...,...,...,...
116747,116747,multiple themes spacer block crashes quick sum...,0
116748,116748,corescript strand of the ancients ship bug 15d...,0
116749,116749,sqs name of missing queue is not reported on a...,1
116750,116750,net stats many received tcp packets count as d...,1


In [3]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,58376
0,58376


In [4]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [5]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [6]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11676
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 93401
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11675
    })
})

In [7]:
# Uncomment this if not logged in.
#notebook_login()

In [8]:
# Push to Hugging Face Hub
ds.push_to_hub(name+"_min_100_issues_per_repo")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/94 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]